In [ ]:
import open3d as o3d
import numpy as np
import copy
import os
import sys

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

# Robust Kernels 

This tutorial demonstrates the use of robust kernels in the context of outlier rejection. For this particular tutorial, we will be using the ICP (Iterative Closest Point) registration algorithm as the _target_ problem where we want to deal with outlier. Even so, the theory applies for any given optimization problem and not just for ICP. For the moment the robust kernels has been only implemented for the `PointToPlane` ICP.

The notation and some of the kernels implemented in `Open3D` has been inspired by the publication "Analysis of Robust Functions for Registration Algorithms" [\[Babin2019\]](../reference.html#babin2019)

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

## Input
The code below reads a source point cloud and a target point cloud from two files. A rough transformation is given.

<div class="alert alert-info">
    
**Note:** 

The initial alignment is usually obtained by a global registration algorithm. See [Global registration](../pipelines/global_registration.rst) for examples.

</div>

In [ ]:
source = o3d.io.read_point_cloud("../../test_data/ICP/cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("../../test_data/ICP/cloud_bin_1.pcd")
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)

## Point-to-plane ICP using Robust Kernels
$$
\def\argmin{\mathop{\rm argmin}}
\newcommand{\mat}[1]{\mathbf{#1}}
\newcommand{\veca}[1]{\vec{#1}}
\renewcommand{\vec}[1]{\mathbf{#1}}
$$
The standard point-to-plane ICP algorithm [\[ChenAndMedioni1992\]](../reference.html#chenandmedioni1992) minimizes this objective function

\begin{equation}
  E(\mathbf{T}) = \sum_{(\mat{p},\mat{q})\in\mathcal{K}}\big((\mat{p} - \mat{T}\mat{q})\cdot\mat{n}_{\mat{p}}\big)^{2},
\end{equation}

where $\mathbf{n}_{\mathbf{p}}$ is the normal of point $\mathbf{p}$ and $\mathcal{K}$ is the correspondence set between the target point cloud $\mathbf{P}$, and source point cloud $\mathbf{Q}$.

If we call $r_i(\mat{T})$ is $i^\mathrm{th}$ the residual, for a given pair of correspondences $(\mat{p},\mat{q})\in\mathcal{K}$ we can rewrite the objective function as.

\begin{equation}
  E(\mathbf{T}) = \sum_{(\mat{p},\mat{q})\in\mathcal{K}}\big((\mat{p} - \mat{T}\mat{q})\cdot\mat{n}_{\mat{p}}\big)^{2} = \sum_{i=1}^{N} \big({r_i(\mat{T})}\big)^2
\end{equation}

The optimization problem above can be also solved using the iteratively reweighted least squares (IRLS) approach, which solves a sequence of weighted least squares problems.

\begin{equation}
  E(\mathbf{T}) =  \sum_{i=1}^{N} w_i \big({r_i(\mat{T})}\big)^2
\end{equation}

### Outlier Rejection with Robust Kernels

The main idea of a robust loss is to downweight large residuals that are assumed to be caused from outliers such that their influence on the solution is reduced. This is achieved by optimizing $E(\mat{T})$ as:

\begin{equation}
  E(\mathbf{T}) = \sum_{(\mat{p},\mat{q})\in\mathcal{K}}\rho\big((\mat{p} - \mat{T}\mat{q})\cdot\mat{n}_{\mat{p}}\big) = \sum_{i=1}^{N} \rho\big({r_i(\mat{T})}\big),
\end{equation}

where $\rho(r)$ is also called the robust loss or kernel.

We can see the relation between the least squares optimization in stanad weighted non-linear least squares and robust loss optimization. By setting the weight $w_i= \frac{1}{r_i(\mat{T})}\rho'(r_i(\mat{T}))$, we can solve the robust loss optimization problem by using the existing techniques for weighted least-squares.

Then we minimize the objective function using Gauss-Newton and determine increments by iteratively solving:

\begin{align}
\left(\mat{J}^\top \mat{W} \mat{J}\right)^{-1}\mat{J}^\top\mat{W}\vec{r},
\end{align}

where $\mat{W} \in \mathbb{R}^{N\times N}$ is a diagonal matrix containing
weights $w_i$ for each residual $r_i$

### How to use it

`registration_icp` is called with a different parameter `TransformationEstimationPointToPlane(loss)`. Where `loss` is a given loss function(also called robust kernel). 

Internally, `TransormationEstimationPointToPlane(loss)` implements functions to compute the weighted residuals and Jacobian matrices of the point-to-plane ICP objective according to the provided robust kernel.

<div class="alert alert-info">
 
**Note:** 

For more details on the ICP algorithm please see [ICP Registration](../pipelines/icp_registration.rst).

</div>

## Add Noise

To better show the impact of the usage of robust kernels in the registration we add some artificially generated noise to the source point cloud.

<div class="alert alert-info">
    
**Note:** 

The parameter `k` used in the Robust Kernels it's usually pick to match the standard deviation of the noise model of the data. In this sense, `k`, is the discriminator between **inlier**/**outlier**. Altough this is not always trivial to define in real world data, for synthetic one, it's easy to fix in order to illustrate the benefits of the robust kernels.
</div>

In [ ]:
def apply_noise(pcd, mu, sigma):
    noisy_pcd = copy.deepcopy(pcd)
    points = np.asarray(noisy_pcd.points)
    points += np.random.normal(mu, sigma, size=points.shape)
    noisy_pcd.points = o3d.utility.Vector3dVector(points)
    return noisy_pcd


# Add Gaussian noise to the input data
mu, sigma = 0, 0.1  # mean and standard deviation
source_noisy = apply_noise(source, mu, sigma)
print("Source PointCloud + noise:")
o3d.visualization.draw_geometries([source_noisy],
                                  zoom=0.4459,
                                  front=[0.9288, -0.2951, -0.2242],
                                  lookat=[1.6784, 2.0612, 1.4451],
                                  up=[-0.3402, -0.9189, -0.1996])

In [ ]:
threshold = 0.02
print("Vanilla point-to-plane ICP, threshold={} :".format(threshold))
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

## Tuning Vanilla ICP

Given the fact that we are now dealing with noise, we need to increase the threshold to search for nearest neighbors.
Without a robust kernel, the traditional ICP has no chance to deal with this outliers in a bigger search space.

In [ ]:
threshold = 1.0
print("Vanilla point-to-plane ICP, threshold={} :".format(threshold))
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

## Reject Noise Outliers

Using the bigger `threshold` (set to `1.0`) + a robust kernel, we can properly register the two `PointClouds`:

<div class="alert alert-info">
    
**Note:** 

For this example we use the **TukeyLoss**, available in `open3d.pipelines.registration.TukeyLoss`. For the parameter `k` we set it to match the std deviation of the noise model $k = \sigma$.

</div>

In [ ]:
print("Robust point-to-plane ICP, threshold={} :".format(threshold))
loss = o3d.pipelines.registration.TukeyLoss(k=sigma)
print("Using robust loss:", loss)
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)